### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
%matplotlib inline

import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten
from keras import backend
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
# Versioning
print(keras.__version__)
print(tf.__version__)

2.2.4
1.10.0


### Read in Normalized Option Pricing Data Set

In [3]:
option_df = pd.read_pickle("opt_data_z.pkl")
## Move data into numpy array
prices = option_df.values

In [4]:
# X values
X = prices[:,:4]

# Y values 
y = prices[:,4:5]

# Use Sklearn library to randomly split data into training/testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Build Keras/Scikit Learn Function

In [ ]:
def create_model(optimizer = 'rmsprop'):

    nodes = 200

    # Create Model
    model = Sequential()

    # Layer 1
    model.add(Dense(nodes, input_shape=(4,))) 
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    # Layer 2
    model.add(Dense(nodes)) 
    model.add(Activation ('relu'))
    model.add(Dropout(0.25))

    # Layer 3
    model.add(Dense(nodes))
    model.add(Activation ('relu'))
    model.add(Dropout(0.25))

    # Layer 4 - Output Layer
    model.add(Dense(1))
    model.add(Activation('elu'))
    
    model.compile(loss='mse', optimizer = optimizer, metrics=['accuracy'])
    
    return model

#### Instantiate Keras Classifier

In [ ]:
neural_network = KerasClassifier(build_fn = create_model, verbose = 0)

#### Define Search Parameters 

In [ ]:
epochs = [5]
batches = [5]
optimizers = ['rmsprop', 'adam']

hyperparameters = dict(optimizer=optimizers, epochs=epochs)

In [ ]:
callbacks_list = [EarlyStopping(monitor = 'loss', patience = 1)]

In [ ]:
# Create grid search
grid = GridSearchCV(estimator = neural_network, param_grid = hyperparameters, cv = 5)

# Fit grid search
grid_result = grid.fit(X_train, y_train, verbose = 2, callbacks= callbacks_list)

Epoch 1/5
 - 125s - loss: 286389304.9738 - acc: 2.3084e-05
Epoch 2/5
 - 140s - loss: 123378579.3130 - acc: 3.2702e-05
Epoch 3/5
 - 50s - loss: 109204601.3572 - acc: 6.3481e-05
Epoch 4/5
 - 50s - loss: 93173775.8881 - acc: 4.6168e-05
Epoch 5/5
 - 51s - loss: 70434038.5417 - acc: 5.7710e-05
Epoch 1/5


In [ ]:
grid_result.best_params_

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=10, validation_split=0.1, verbose = 2, callbacks= callbacks_list)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
plt.figure(figsize = (15,10))
plt.scatter(y_test, predictions)
plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")
plt.plot([0,1], [0,1], 'r')
plt.grid(True)

